In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import datetime as dt
tf.random.set_seed(0)

In [16]:
data = pd.read_csv(r"D:\Data Science\datasets\yahoo_data.csv")
data['Date']=pd.to_datetime(data['Date'])
#data['Date_val'] = data['Date'].map(dt.datetime.toordinal)
data.drop('Unnamed: 0',inplace=True,axis=1)
data.head()

Date      Open      High       Low    Close*  Adj Close**       Volume
0 2023-04-28  33797.43  34104.56  33728.40  34098.16     34098.16  354310000.0
1 2023-04-27  33381.66  33859.75  33374.65  33826.16     33826.16  343240000.0
2 2023-04-26  33596.34  33645.83  33235.85  33301.87     33301.87  321170000.0
3 2023-04-25  33828.34  33875.49  33525.39  33530.83     33530.83  297880000.0
4 2023-04-24  33805.04  33891.15  33726.09  33875.40     33875.40  252020000.0

In [17]:
dataset_summary = pd.DataFrame({"Date" : "The date on which trade occured", "Open" : "Day's Opening trading price", "High" : "Day's highest trading price",
                   "Low" : "Day's lowest trading price", "Close*" : "Day's Closing trading price", "Adj Close**" : "Day's Closing trading price after adjustustments",
                   "Volume" : "Quantity of stocks sold between open and close"},index=["Description"])
display(pd.DataFrame.transpose(dataset_summary))

Description
Date                          The date on which trade occured
Open                              Day's Opening trading price
High                              Day's highest trading price
Low                                Day's lowest trading price
Close*                            Day's Closing trading price
Adj Close**  Day's Closing trading price after adjustustments
Volume         Quantity of stocks sold between open and close

In [18]:
# #sns.heatmap(data.corr(),annot=True)
# fig = plt.figure(figsize=(15,10))

# for i in range(1,len(data.columns)):    
#     plt.subplot(3,2,i)
#     plt.scatter(data['Date'],data[data.columns[i]])
#     plt.xlabel('Date')
#     plt.ylabel(data.columns[i])
# plt.show()

In [19]:
open_price = data[['High']].values.astype('float32')
ss = MinMaxScaler(feature_range=(0,1))
open_price = ss.fit_transform(open_price)
#plt.plot(open_price)

In [20]:
train_size = round(len(open_price)*0.67)
test_size = len(open_price) - train_size
train, test = open_price[:train_size], open_price[train_size:len(open_price)]

In [21]:
def create_dataset(dataset, lookback):
    X, y = [], []

    for i in range(len(dataset)-lookback-1):
        X.append(dataset[i:i+lookback])
        y.append(dataset[i+lookback])
    
    return np.array(X), np.array(y)

In [28]:
lookback = 8
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(834, 8, 1) (834, 1)
(406, 8, 1) (406, 1)


array([[0.84027886],
       [0.8265499 ],
       [0.81455314],
       [0.8274325 ],
       [0.8283107 ],
       [0.82649827],
       [0.827427  ],
       [0.83205867]], dtype=float32)

In [23]:
model = Sequential()
model.add(LSTM(4,input_shape=(lookback,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(X_train,y_train,epochs=25,batch_size=1,verbose=2)

Epoch 1/25
834/834 - 10s - loss: 0.0214 - 10s/epoch - 12ms/step
Epoch 2/25
834/834 - 5s - loss: 0.0011 - 5s/epoch - 6ms/step
Epoch 3/25
834/834 - 5s - loss: 0.0011 - 5s/epoch - 6ms/step
Epoch 4/25
834/834 - 5s - loss: 0.0010 - 5s/epoch - 6ms/step
Epoch 5/25
834/834 - 6s - loss: 0.0010 - 6s/epoch - 7ms/step
Epoch 6/25
834/834 - 5s - loss: 9.5325e-04 - 5s/epoch - 6ms/step
Epoch 7/25
834/834 - 6s - loss: 8.8924e-04 - 6s/epoch - 7ms/step
Epoch 8/25
834/834 - 6s - loss: 8.1689e-04 - 6s/epoch - 7ms/step
Epoch 9/25
834/834 - 5s - loss: 8.2937e-04 - 5s/epoch - 7ms/step
Epoch 10/25
834/834 - 6s - loss: 7.1900e-04 - 6s/epoch - 7ms/step
Epoch 11/25
834/834 - 6s - loss: 7.0033e-04 - 6s/epoch - 7ms/step
Epoch 12/25
834/834 - 17s - loss: 6.2162e-04 - 17s/epoch - 20ms/step
Epoch 13/25
834/834 - 6s - loss: 5.9179e-04 - 6s/epoch - 7ms/step
Epoch 14/25
834/834 - 4s - loss: 5.7215e-04 - 4s/epoch - 5ms/step
Epoch 15/25
834/834 - 4s - loss: 5.5421e-04 - 4s/epoch - 5ms/step
Epoch 16/25
834/834 - 4s - loss: 

In [24]:
train_pred = model(X_train)
# print(train_pred.shape)
# train_pred = train_pred[:,-1]
test_pred = model(X_test)
#print(test_pred.shape)
# test_pred = test_pred[:,-1]
#print(train_pred.shape)
train_pred = ss.inverse_transform(train_pred)
test_pred = ss.inverse_transform(test_pred)
y_train = ss.inverse_transform(y_train)
y_test = ss.inverse_transform(y_test)
#print(train_pred.shape, y_train.shape)
train_acc = np.sqrt(mean_squared_error(y_train,train_pred))
test_acc = np.sqrt(mean_squared_error(y_test,test_pred))
print(r2_score(y_train,train_pred))
print(r2_score(y_test,test_pred))

0.9910200187872247
0.9615242648303055


In [25]:
# shift train predictions for plotting
train_plot = np.ones_like(open_price) * np.nan
# print(len(train_pred))
# print(train_plot.shape)
# print(train_pred.shape)
open_price = ss.inverse_transform(open_price)
train_plot[lookback:train_size-1] = train_pred
# shift test predictions for plotting
test_plot = np.ones_like(open_price) * np.nan
test_plot[train_size+lookback:len(open_price)-1] = test_pred
# plot
plt.plot(open_price, c='b')
plt.plot(train_plot, c='r')
plt.plot(test_plot, c='g')
#print(test_plot[75:])
#print(open_price)
plt.show()

In [26]:
open_price[:5], train_pred[:5]

(array([[34104.56],
        [33859.75],
        [33645.83],
        [33875.49],
        [33891.15]], dtype=float32),
 array([[33937.62758331],
        [34002.11804222],
        [33987.26154578],
        [34063.41251568],
        [34050.2183143 ]]))

## Appendix

https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/